Data Science Gradio Image Filtering<br>


In [1]:
import gradio as gr
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
import cv2
from PIL import Image
from scipy  import ndimage

<h3>Grayscale an Image</h3>

Formula: Grayscale = 0.299R + 0.587G + 0.114B <br>
Source: https://www.dynamsoft.com/blog/insights/image-processing/image-processing-101-color-space-conversion/

In [3]:
Grayscale = [0.299, 0.587,  0.114]
def grayscale(img):

    # multiply with weights 
    red = np.array(img[:, :, 0]) * Grayscale[0] # 0 = first channel i.e., red
    green = np.array(img[:, :, 1]) * Grayscale[1] # 1 = second channel i.e., green
    blue = np.array(img[:, :, 2]) * Grayscale[2] # 2 = third channel i.e., blue
    
    # add the weights
    sum = (red+green+blue)

    grayImage = np.zeros(img.shape) # make a new image with the same shape as 'img' filled w zeroes
    grayImage = img.copy() # copy the original image 

    # now, change the color channels to gray
    for i in range(3):
        grayImage[:,:,i] = sum

    return grayImage


In [4]:
demo = gr.Interface(fn=grayscale,
                    inputs="image", 
                    outputs="image",
                    title="Grayscale an Image",
                    description="Upload an image and grayscale it.",
                    theme=gr.themes.Soft())
demo.launch() 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


<h3>Blur an Image</h3>

Gaussian blur <br>
 - reduce image noise and smooth out details of an image
 - working
   averages the pixel values in the image using a Gaussian function
 - Gaussian function : 
     - bell curve shape with a peak at the center and lower values on either side
     - applying this function to each pixel in an image assigns a higher weight to the pixel values close to the center and lower weights to the pixel values farther from the center.
 - How?
      - reducing the sharpness of edges
      - reduce the fine details in the image
      - reduces the contrast between different parts of the image
      - reduces the sharpness of transitions between different colors or intensities
 - ![This is a alt text.](https://i.stack.imgur.com/U7vYB.gif "This is a sample image.")

In [5]:
def gaussianKernel(kernel_size, sigma):
    x, y = np.mgrid[-kernel_size//2 + 1 : kernel_size//2 + 1, -kernel_size//2 + 1:kernel_size//2 + 1] # create a 2d grid and store the coordinates (x,y) of each pixel in this grid
    # applying formula
    kernel =   np.exp(-((x**2 + y**2) / (2.0*sigma**2))) 
    kernel = (1/(2*np.pi*(sigma)**2)) * kernel
    return kernel


def blur_img( Sigma,  Image):
    kernel = gaussianKernel(11, Sigma)
    Image = cv2.resize(Image, (512,512)) # resize the image 
    img_blurred = cv2.filter2D(Image,-1,kernel) #convolve: dot product of kernel and each pixel of the image,  -1 : output should also be image

    return img_blurred


In [6]:
demo = gr.Interface(fn=blur_img,
                    inputs=[ gr.Slider(1, 10, value=3, step=1 ,label="Sigma", info="Choose betwen 1 and 10"),  "image"], 
                    outputs="image",
                    title="Blur an Image with Gaussian Blur",
                    description="Upload an image and blur it.",
                    theme=gr.themes.Soft())
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


<h3>Canny Edge Detection</h3>

Step1: Gaussian Smoothing

In [22]:
def gaussiankernel(kernel_size, sigma):
    x, y = np.mgrid[-kernel_size//2 + 1:kernel_size//2 + 1, -kernel_size//2 + 1:kernel_size//2 + 1]
    kernel =   np.exp(-((x**2 + y**2) / (2.0*sigma**2))) 
    kernel = (1/(2*np.pi*(sigma)**2)) * kernel
    return kernel

def gaussianFilter(img, kernel_size, sigma):
    img = cv2.resize(np.array(img), (512,512)) # resize image
    img = grayscale(img) # grayscale the image, because gaussian filter is mostly to grayscale images

    kernel = gaussiankernel(kernel_size, sigma) # find kernel
    img_n = img[:,:,0] #  convert to 2d 
    img_smoothed =  ndimage.convolve(img_n,kernel) # apply the kernel
    return img_smoothed
    

Step 2: Intensity gradients

In [23]:
# applying sobel filters which detect changes in intensity between neighboring pixels in an image, computes gradient of the image intensity in the x and y directions
def intensity_gradient(img_smoothed): #  computes the magnitude and direction of the image gradient using the Sobel filters.
    x = cv2.Sobel(img_smoothed, cv2.CV_64F, 1, 0, ksize=3) # CV_64F: output data type of image,1: for x, 0: for y, ksize = 3x3
    y = cv2.Sobel(img_smoothed, cv2.CV_64F, 0, 1, ksize=3)

    G = (x ** 2 + y ** 2) ** 0.5 # magnitude
    theta = np.rad2deg(np.arctan2(y, x)) # direction
    return G, theta


Step 3: Non-maximum suppression

In [24]:
# non maximum suppression thins out edges and highlight only the most prominent ones
def non_max_suppression(G, theta):
    direction = theta 
    direction[direction < 0] += 180
    r, c = G.shape[:2]
    NMS = np.zeros((r, c), dtype=np.int32)
    
    #for each pixel in the image,check whether its gradient magnitude is the maximum along the gradient direction. If it is, the pixel is kept, otherwise it is set to zero
    for i in range(1,r-1):
        for j in range(1,c-1):
            pix_1 = 255
            pix_2 = 255
            if (0 <= direction[i,j] < 22.5) or (157.5 <= direction[i,j] <= 180):
                pix_1 = G[i, j+1] # right
                pix_2 = G[i, j-1] # left
            
            elif (22.5 <= direction[i,j] < 67.5):
                pix_1 = G[i+1, j-1] # down and left
                pix_2 = G[i-1, j+1] # up and right
            
            elif (67.5 <= direction[i,j] < 112.5):
                pix_1 = G[i+1, j] # down
                pix_2 = G[i-1, j] # up
            
            elif (112.5 <= direction[i,j] < 157.5):
                pix_1 = G[i-1, j-1] # above and left
                pix_2 = G[i+1, j+1] # down and right

            if (G[i,j] >= pix_1) and (G[i,j] >= pix_2): #if gradient magnitude is the maximum along the gradient direction
                NMS[i,j] = G[i,j]
            else:
                NMS[i,j] = 0
    return NMS

Step 4: Double Threshhold

In [29]:
def double_threshold(NMS, high_thresh, low_thresh, weak_pixel, strong_pixel):
    highThreshold = NMS.max() * high_thresh; # max val in NMS * high thresh
    lowThreshold = highThreshold * low_thresh;

    m,n = NMS.shape
    double_thresh = np.zeros((m,n), dtype=np.int32)

    i, j = np.where(NMS >= highThreshold)
    strong = np.int32(strong_pixel)
    double_thresh[i, j] = strong
    
    i, j = np.where((NMS <= highThreshold) & (NMS >= lowThreshold))
    weak = np.int32(weak_pixel)
    double_thresh[i, j] = weak
    
    return double_thresh

Step 5: Edge tracking by hysteresis

In [26]:
#  form continuous curves or lines
def hysteresis(double_thresh, weak_pixel, strong_pixel):
    r, c  = double_thresh.shape

    # for each pixel in the double threshold image
    for i in range(1, r-1):
        for j in range(1, c-1): # If at least one of the neighboring 8 pixels is a strong pixel, the weak pixel is promoted to a strong pixel
            if (double_thresh[i,j] == weak_pixel):
                if ((double_thresh[i+1, j-1] == strong_pixel) or (double_thresh[i+1, j] == strong_pixel) or (double_thresh[i+1, j+1] == strong_pixel) or (double_thresh[i, j-1] == strong_pixel) or (double_thresh[i, j+1] == strong_pixel)or (double_thresh[i-1, j-1] == strong_pixel) or (double_thresh[i-1, j] == strong_pixel) or (double_thresh[i-1, j+1] == strong_pixel)):
                    double_thresh[i, j] = strong_pixel
                else:
                    double_thresh[i, j] = 0
    return double_thresh
                

In [27]:
def display(img):
    i = img.astype(np.uint8)
    cv2.imshow('Original', i)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def canny_detection(img, sigma, low_thresh, high_thresh, step):
    if step == "":
        raise gr.Error("Select a Step!")
    if step == "Input Image":
        return grayscale(img)
    elif step == "Gaussian Smoothing":
        return gaussianFilter(img, 11, sigma)
    elif step == "Intensity Gradient: Magnitude":
        img_smoothed = gaussianFilter(img, 11, sigma)
        G, theta = intensity_gradient(img_smoothed)
        return G.astype(np.uint8)
    elif step == "Intensity Gradient: Theta":
        img_smoothed = gaussianFilter(img, 11, sigma)
        G, theta = intensity_gradient(img_smoothed)
        return theta.astype(np.uint8)
    elif step == "Non-maximum Suppression":
        img_smoothed = gaussianFilter(img, 11, sigma)
        G, theta = intensity_gradient(img_smoothed)
        return  non_max_suppression(G, theta)
    elif step == "Double Threshold":
        img_smoothed = gaussianFilter(img, 11, sigma)
        G, theta = intensity_gradient(img_smoothed)
        Z = non_max_suppression(G, theta)
        return double_threshold(Z, low_thresh, high_thresh, 120, 255)
    elif step == "Hysteresis threshold":
        img_smoothed = gaussianFilter(img, 11, sigma)
        G, theta = intensity_gradient(img_smoothed)
        Z = non_max_suppression(G, theta)
        R = double_threshold(Z, low_thresh, high_thresh, 120, 255)
        return hysteresis(R, 120, 255)
    return img

In [30]:
demo = gr.Interface(fn=canny_detection,
                    inputs=
                    ["image",
                     gr.Slider(1, 10, value=3, step=1 ,label="Sigma", info="Choose betwen 1 and 10"), 
                     gr.Slider(0, 1, value=0.15, step=0.01 ,label="Low Threshold"),
                     gr.Slider(0, 1, value=0.50, step=0.01 ,label="High Threshold"),                     
                     gr.Dropdown(["Input Image","Gaussian Smoothing", "Intensity Gradient: Magnitude", "Intensity Gradient: Theta", "Non-maximum Suppression", "Double Threshold", "Hysteresis threshold"], label="Step", info="Choose a step from Canny Edge Detection.")
                    ], 
                    outputs="image",
                    title="Canny Edge Detection",
                    description="This demonstration shows the 5 steps of the classical Canny edge detector.",
                    theme=gr.themes.Soft())


demo.launch() 

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


<h3>Invert Colors of an Image</h3>

In [14]:
def invert(image):
    rgb_channels = image[:,:,:3] # contains all the rows and columns of the original image array, but only the first three channels (i.e., the RGB channels)
    
    # subtract 255 from each pixel to invert the image
    inverted_image = 255 - rgb_channels
    inverted_image = Image.fromarray(inverted_image)
    return inverted_image

In [15]:
demo = gr.Interface(fn=invert,
                    inputs="image", 
                    outputs="image",
                    title="Invert colors of an image",
                    description="Upload an image and invert its colors.",
                    theme=gr.themes.Soft())
demo.launch() 

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
